First stage - playing with Arango
1) With nebula API

In [44]:
import sys
sys.path.append("../") 
from nebula_api.nebula_enrichment_api import NRE_API
nre = NRE_API()
db = nre.db
print(nre.db_host)

http://ec2-18-158-123-0.eu-central-1.compute.amazonaws.com:8529/


2) Define database without API

In [45]:
from arango import ArangoClient
client = ArangoClient(hosts='http://ec2-18-158-123-0.eu-central-1.compute.amazonaws.com:8529')
dbname = 'nebula_development'
db_manual = client.db(dbname, username='nebula', password='nebula')
print(db_manual)

<StandardDatabase nebula_development>


Select movies from database
Remove FILTER doc._id == \'Movies/114208744\' from query for all movies
#FOR doc IN Movies RETURN doc
ArangoUI:
http://ec2-18-158-123-0.eu-central-1.compute.amazonaws.com:8529

In [46]:
nebula_movies = []
query = 'FOR doc IN Movies FILTER doc._id == \'Movies/114208744\' RETURN doc'
cursor = db_manual.aql.execute(query)
for data in cursor: 
    print(data)
    nebula_movies.append(data)
print(nebula_movies)

{'_key': '114208744', '_id': 'Movies/114208744', '_rev': '_dEl7OJu---', 'movie_id': '03e53e8e2b4a45759ac74b684d6c737d', 'file_name': '1010_TITANIC_00_41_32_072-00_41_40_196.avi', 'movie_name': '1010_TITANIC_00_41_32_072-00_41_40_196', 'description': '1010_TITANIC_00_41_32_072-00_41_40_196', 'tags': ['lsmdc', 'pegasus', 'visual genome'], 'full_path': '/dataset/development/1010_TITANIC_00_41_32_072-00_41_40_196.mp4', 'url_path': 'static/development/1010_TITANIC_00_41_32_072-00_41_40_196.mp4', 'last_frame': 300, 'scenes': [[0, 195]], 'scene_elements': [[0, 29], [29, 100], [100, 195]], 'mdfs': [[2, 14, 27], [31, 64, 98], [102, 147, 193]], 'meta': {'fps': 23, 'height': 1080, 'width': 1920}, 'updates': 4, 'status': 'created'}
[{'_key': '114208744', '_id': 'Movies/114208744', '_rev': '_dEl7OJu---', 'movie_id': '03e53e8e2b4a45759ac74b684d6c737d', 'file_name': '1010_TITANIC_00_41_32_072-00_41_40_196.avi', 'movie_name': '1010_TITANIC_00_41_32_072-00_41_40_196', 'description': '1010_TITANIC_00_41

Let's see the movie:

In [47]:
movie_file = nebula_movies[0]['url_path']
print(movie_file)

static/development/1010_TITANIC_00_41_32_072-00_41_40_196.mp4


In [48]:
from IPython.display import HTML
server = "http://ec2-52-57-66-3.eu-central-1.compute.amazonaws.com:7000/"
HTML("""
    <video alt="test" controls>
        <source src="{}" type="video/mp4">
    </video>
""".format(server+movie_file))                    


Get scene elements of the movie

In [7]:
movie_id = 'Movies/114208744'
query = 'FOR doc IN StoryLine FILTER doc.arango_id == "{}"  RETURN doc'.format(movie_id)
cursor = db.aql.execute(query)
#print("Movie: ", m)
all_scenes = []
scene = {}
for i, data in enumerate(cursor):
    #print("SCENE ELEM.: ", data['scene_'])
    scene = {'scene_graph_triplets': data['scene_graph_triplets'], 'sentences': data['sentences'], \
            'movie_id': data['movie_id'], 'arango_id': data['arango_id'], \
            'description': data['description'], 'scene_element': data['scene_element'], 'start': data['start'], 'stop': data['stop'], \
                }
    print(scene)
    all_scenes.append(scene)
print(all_scenes)

{'scene_graph_triplets': [[['scene', 'shows', 'camera strap'], ['scene', 'features', 'yacht'], ['clothes', 'on', 'ship']], [['scene', 'features', 'yacht'], ['scene', 'shows', 'camera strap'], ['clothes', 'on', 'ship']], [['clothes', 'on', 'ship'], ['scene', 'features', 'yacht'], ['boat', 'carrie', 'people']]], 'sentences': [['The swim dress is titanic.', 'The titanic dress is amazing.', 'Wear titanic dress.'], ['The swim dress is titanic.', 'The titanic dress is amazing.', 'Wear titanic dress.'], ['The swim dress is titanic.', 'The titanic dress is amazing.', 'Wear titanic dress.']], 'movie_id': '03e53e8e2b4a45759ac74b684d6c737d', 'arango_id': 'Movies/114208744', 'description': '', 'scene_element': 0, 'start': 0, 'stop': 29}
{'scene_graph_triplets': [[['scene', 'shows', 'dove'], ['scene', 'shows', 'pan'], ['hand', 'on back of', 'motorcycle']], [['scene', 'features a', 'teddy bear'], ['scene', 'shows', 'dove'], ['movie', 'standing on top of', 'desk']], [['scene', 'shows', 'pan'], ['this

Get different detections for movie

In [8]:
movie_id = 'Movies/114208744'
query = 'FOR doc IN Nodes FILTER doc.arango_id == "{}"  RETURN doc'.format(movie_id)
cursor = db.aql.execute(query)
#print("Movie: ", m)
all_scenes = []
scene = {}
for i, data in enumerate(cursor):
    print(data)

{'_key': '114401559', '_id': 'Nodes/114401559', '_rev': '_dCsQgOW---', 'movie_id': '03e53e8e2b4a45759ac74b684d6c737d', 'arango_id': 'Movies/114208744', 'actor_id': None, 'description': 'stand', 'frame_id': '0', 'class': 'Actions', 'confidence': 0.8791552782058716, 'updates': 1, 'box': [0.3364400565624237, 0.1250765025615692, 0.6431668996810913, 0.8714401125907898]}
{'_key': '114401564', '_id': 'Nodes/114401564', '_rev': '_dCsQgT----', 'movie_id': '03e53e8e2b4a45759ac74b684d6c737d', 'arango_id': 'Movies/114208744', 'actor_id': None, 'description': 'listen to (a person)', 'frame_id': '0', 'class': 'Actions', 'confidence': 0.543153703212738, 'updates': 1, 'box': [0.3364400565624237, 0.1250765025615692, 0.6431668996810913, 0.8714401125907898]}
{'_key': '114401569', '_id': 'Nodes/114401569', '_rev': '_dCsQgXq---', 'movie_id': '03e53e8e2b4a45759ac74b684d6c737d', 'arango_id': 'Movies/114208744', 'actor_id': None, 'description': 'carry/hold (an object)', 'frame_id': '0', 'class': 'Actions', 'c